In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import re
import string
from time import time  # To time our operations
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from gensim.models.phrases import Phrases, Phraser
import multiprocessing
from gensim.models import Word2Vec

['file.txt']


In [ ]:
# Combine provided two files
filenames = ['neg.txt', 'pos.txt']
with open('../input/file', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [17]:
# Function to remove Stopwords
def remove_stopwords(line):    
    stop_words = set(stopwords.words('english'))
    line = [w for w in line if not w in stop_words]
    return line

#Function to remove Punctuation and normalize characters to lowercase
def remove_punc_lower(line):
    line="".join([char for char in line if char not in string.punctuation])
    line=line.lower()
    return line

# Function to Tokenize words
def tokenize(line):
    tokens = re.split('\W+', line)
    return tokens


if __name__ == "__main__":
    input_path = '../input/file.txt'
    f=open(input_path)
    text=f.read().split('\n')
    text_with_stop=[]
    for i in range (len(pos)):
        p=tokenize(remove_punc_lower(text[i]))
        text_with_stop.append(p)

Gensim Phrases package was used to automatically detect common phrases (bigrams) from a list of sentences.
https://radimrehurek.com/gensim/models/phrases.html

The main reason is to catch words like "mr_burns" or "bart_simpson".

In [18]:
# Creates the relevant phrases from the list of sentences:
phrases = Phrases(text_with_stop, min_count=30, progress_per=10000)
# The goal of Phraser() is to cut down memory consumption of Phrases()
# by discarding model state not strictly needed for the bigram detection task
bigram = Phraser(phrases)
#Transform the corpus based on the bigrams detected
sentences = bigram[text_with_stop]

INFO - 02:27:38: collecting all words and their counts
INFO - 02:27:38: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 02:27:38: PROGRESS: at sentence #10000, processed 151664 words and 72705 word types
INFO - 02:27:39: PROGRESS: at sentence #20000, processed 303579 words and 120774 word types
INFO - 02:27:39: PROGRESS: at sentence #30000, processed 454294 words and 161859 word types
INFO - 02:27:40: PROGRESS: at sentence #40000, processed 605215 words and 201814 word types
INFO - 02:27:40: PROGRESS: at sentence #50000, processed 755236 words and 236697 word types
INFO - 02:27:40: PROGRESS: at sentence #60000, processed 904909 words and 266934 word types
INFO - 02:27:41: PROGRESS: at sentence #70000, processed 1054212 words and 294425 word types
INFO - 02:27:41: PROGRESS: at sentence #80000, processed 1203785 words and 325067 word types
INFO - 02:27:42: PROGRESS: at sentence #90000, processed 1354427 words and 361736 word types
INFO - 02:27:42: PROGRESS: at sentenc

## Gensim Word2Vec Implementation:


In [19]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

## Building the Vocabulary Table:
Word2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them):

In [20]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 02:28:35: collecting all words and their counts
INFO - 02:28:35: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 02:28:35: PROGRESS: at sentence #10000, processed 146480 words, keeping 10869 word types
INFO - 02:28:36: PROGRESS: at sentence #20000, processed 292990 words, keeping 15688 word types
INFO - 02:28:37: PROGRESS: at sentence #30000, processed 438761 words, keeping 19568 word types
INFO - 02:28:37: PROGRESS: at sentence #40000, processed 584671 words, keeping 23447 word types
INFO - 02:28:38: PROGRESS: at sentence #50000, processed 729583 words, keeping 26672 word types
INFO - 02:28:39: PROGRESS: at sentence #60000, processed 874145 words, keeping 29398 word types
INFO - 02:28:39: PROGRESS: at sentence #70000, processed 1018523 words, keeping 31773 word types
INFO - 02:28:40: PROGRESS: at sentence #80000, processed 1163503 words, keeping 34655 word types
INFO - 02:28:41: PROGRESS: at sentence #90000, processed 1310640 words, keeping 38016 word t

INFO - 02:29:27: effective_min_count=20 leaves 11383316 word corpus (97% of original 11709780, drops 326464)
INFO - 02:29:27: deleting the raw counts dictionary of 165967 items
INFO - 02:29:27: sample=6e-05 downsamples 668 most-common words
INFO - 02:29:27: downsampling leaves estimated 4511129 word corpus (39.6% of prior 11383316)
INFO - 02:29:27: estimated required memory for 15529 words and 300 dimensions: 45034100 bytes
INFO - 02:29:27: resetting layer weights


Time to build vocab: 0.88 mins


## Training of the model:

In [21]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 02:29:27: training model with 3 workers on 15529 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 02:29:29: EPOCH 1 - PROGRESS: at 1.70% examples, 75269 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:30: EPOCH 1 - PROGRESS: at 3.50% examples, 76646 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:31: EPOCH 1 - PROGRESS: at 5.21% examples, 75952 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:32: EPOCH 1 - PROGRESS: at 6.94% examples, 75432 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:33: EPOCH 1 - PROGRESS: at 8.67% examples, 74668 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:34: EPOCH 1 - PROGRESS: at 10.47% examples, 74803 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:35: EPOCH 1 - PROGRESS: at 12.17% examples, 75122 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:36: EPOCH 1 - PROGRESS: at 13.94% examples, 75471 words/s, in_qsize 0, out_qsize 0
INFO - 02:29:37: EPOCH 1 - PROGRESS: at 15.72% examples, 75705 words/s, in_qsize 0, out_qsize 0
INFO -

INFO - 02:30:54: EPOCH 2 - PROGRESS: at 43.29% examples, 76478 words/s, in_qsize 0, out_qsize 0
INFO - 02:30:55: EPOCH 2 - PROGRESS: at 44.94% examples, 76416 words/s, in_qsize 0, out_qsize 0
INFO - 02:30:56: EPOCH 2 - PROGRESS: at 46.60% examples, 76414 words/s, in_qsize 0, out_qsize 0
INFO - 02:30:57: EPOCH 2 - PROGRESS: at 48.24% examples, 76389 words/s, in_qsize 0, out_qsize 0
INFO - 02:30:58: EPOCH 2 - PROGRESS: at 49.89% examples, 76483 words/s, in_qsize 0, out_qsize 0
INFO - 02:31:00: EPOCH 2 - PROGRESS: at 51.70% examples, 76473 words/s, in_qsize 0, out_qsize 0
INFO - 02:31:01: EPOCH 2 - PROGRESS: at 53.43% examples, 76441 words/s, in_qsize 0, out_qsize 0
INFO - 02:31:02: EPOCH 2 - PROGRESS: at 55.12% examples, 76473 words/s, in_qsize 0, out_qsize 0
INFO - 02:31:03: EPOCH 2 - PROGRESS: at 56.77% examples, 76352 words/s, in_qsize 0, out_qsize 0
INFO - 02:31:04: EPOCH 2 - PROGRESS: at 58.49% examples, 76376 words/s, in_qsize 0, out_qsize 0
INFO - 02:31:05: EPOCH 2 - PROGRESS: at 

INFO - 02:32:19: EPOCH 3 - PROGRESS: at 86.40% examples, 76333 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:20: EPOCH 3 - PROGRESS: at 88.18% examples, 76318 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:21: EPOCH 3 - PROGRESS: at 89.99% examples, 76268 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:22: EPOCH 3 - PROGRESS: at 91.77% examples, 76240 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:23: EPOCH 3 - PROGRESS: at 93.50% examples, 76188 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:24: EPOCH 3 - PROGRESS: at 95.21% examples, 76132 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:26: EPOCH 3 - PROGRESS: at 96.92% examples, 76033 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:27: EPOCH 3 - PROGRESS: at 98.72% examples, 76021 words/s, in_qsize 0, out_qsize 0
INFO - 02:32:27: worker thread finished; awaiting finish of 2 more threads
INFO - 02:32:27: worker thread finished; awaiting finish of 1 more threads
INFO - 02:32:27: worker thread finished; awaiting finish of 0 more threads
INFO - 

INFO - 02:33:42: EPOCH 5 - PROGRESS: at 25.59% examples, 74723 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:43: EPOCH 5 - PROGRESS: at 27.29% examples, 74854 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:44: EPOCH 5 - PROGRESS: at 29.06% examples, 75125 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:45: EPOCH 5 - PROGRESS: at 30.75% examples, 75344 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:46: EPOCH 5 - PROGRESS: at 32.54% examples, 75653 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:47: EPOCH 5 - PROGRESS: at 34.31% examples, 75913 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:48: EPOCH 5 - PROGRESS: at 35.93% examples, 75746 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:49: EPOCH 5 - PROGRESS: at 37.64% examples, 75711 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:50: EPOCH 5 - PROGRESS: at 39.35% examples, 75688 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:51: EPOCH 5 - PROGRESS: at 41.07% examples, 75597 words/s, in_qsize 0, out_qsize 0
INFO - 02:33:52: EPOCH 5 - PROGRESS: at 

INFO - 02:35:07: EPOCH 6 - PROGRESS: at 69.12% examples, 76715 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:08: EPOCH 6 - PROGRESS: at 70.85% examples, 76667 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:09: EPOCH 6 - PROGRESS: at 72.60% examples, 76613 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:10: EPOCH 6 - PROGRESS: at 74.32% examples, 76634 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:11: EPOCH 6 - PROGRESS: at 76.13% examples, 76701 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:12: EPOCH 6 - PROGRESS: at 77.85% examples, 76670 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:13: EPOCH 6 - PROGRESS: at 79.56% examples, 76679 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:14: EPOCH 6 - PROGRESS: at 81.28% examples, 76665 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:15: EPOCH 6 - PROGRESS: at 83.09% examples, 76693 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:16: EPOCH 6 - PROGRESS: at 84.87% examples, 76708 words/s, in_qsize 0, out_qsize 0
INFO - 02:35:17: EPOCH 6 - PROGRESS: at 

INFO - 02:36:29: EPOCH 8 - PROGRESS: at 8.58% examples, 74472 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:30: EPOCH 8 - PROGRESS: at 10.22% examples, 73876 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:31: EPOCH 8 - PROGRESS: at 11.92% examples, 74285 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:32: EPOCH 8 - PROGRESS: at 13.69% examples, 74736 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:33: EPOCH 8 - PROGRESS: at 15.47% examples, 75156 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:34: EPOCH 8 - PROGRESS: at 17.15% examples, 75508 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:35: EPOCH 8 - PROGRESS: at 18.87% examples, 75553 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:36: EPOCH 8 - PROGRESS: at 20.61% examples, 75253 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:37: EPOCH 8 - PROGRESS: at 22.40% examples, 75680 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:39: EPOCH 8 - PROGRESS: at 24.18% examples, 75990 words/s, in_qsize 0, out_qsize 0
INFO - 02:36:40: EPOCH 8 - PROGRESS: at 2

INFO - 02:37:54: EPOCH 9 - PROGRESS: at 51.27% examples, 75993 words/s, in_qsize 0, out_qsize 0
INFO - 02:37:55: EPOCH 9 - PROGRESS: at 53.00% examples, 76042 words/s, in_qsize 0, out_qsize 0
INFO - 02:37:56: EPOCH 9 - PROGRESS: at 54.69% examples, 76104 words/s, in_qsize 0, out_qsize 0
INFO - 02:37:57: EPOCH 9 - PROGRESS: at 56.51% examples, 76214 words/s, in_qsize 0, out_qsize 0
INFO - 02:37:58: EPOCH 9 - PROGRESS: at 58.31% examples, 76283 words/s, in_qsize 0, out_qsize 0
INFO - 02:37:59: EPOCH 9 - PROGRESS: at 60.03% examples, 76305 words/s, in_qsize 0, out_qsize 0
INFO - 02:38:00: EPOCH 9 - PROGRESS: at 61.83% examples, 76348 words/s, in_qsize 0, out_qsize 0
INFO - 02:38:01: EPOCH 9 - PROGRESS: at 63.54% examples, 76386 words/s, in_qsize 0, out_qsize 0
INFO - 02:38:02: EPOCH 9 - PROGRESS: at 65.27% examples, 76360 words/s, in_qsize 0, out_qsize 0
INFO - 02:38:04: EPOCH 9 - PROGRESS: at 67.08% examples, 76435 words/s, in_qsize 0, out_qsize 0
INFO - 02:38:05: EPOCH 9 - PROGRESS: at 

INFO - 02:39:19: EPOCH 10 - PROGRESS: at 94.44% examples, 75767 words/s, in_qsize 0, out_qsize 0
INFO - 02:39:20: EPOCH 10 - PROGRESS: at 96.15% examples, 75687 words/s, in_qsize 0, out_qsize 0
INFO - 02:39:21: EPOCH 10 - PROGRESS: at 97.87% examples, 75611 words/s, in_qsize 0, out_qsize 0
INFO - 02:39:22: EPOCH 10 - PROGRESS: at 99.58% examples, 75538 words/s, in_qsize 0, out_qsize 0
INFO - 02:39:23: worker thread finished; awaiting finish of 2 more threads
INFO - 02:39:23: worker thread finished; awaiting finish of 1 more threads
INFO - 02:39:23: worker thread finished; awaiting finish of 0 more threads
INFO - 02:39:23: EPOCH - 10 : training on 11709780 raw words (4510043 effective words) took 59.7s, 75522 effective words/s
INFO - 02:39:23: training on a 117097800 raw words (45109589 effective words) took 595.3s, 75781 effective words/s


Time to train the model: 9.92 mins


# Exploring the model

Use genism library to get the most similar words to a given word. Find 20 most similar
words to “good” and “bad”. Are the words most similar to “good” positive, and words
most similar to “bad” negative? 

In [22]:
w2v_model.most_similar(positive=['good'], topn=20)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
INFO - 02:39:23: precomputing L2-norms of word weight vectors


[('great', 0.7057247161865234),
 ('decent', 0.6706036329269409),
 ('fantastic', 0.5727558732032776),
 ('excellent', 0.5515622496604919),
 ('nice', 0.5501905679702759),
 ('an_excellent', 0.5447955131530762),
 ('terrific', 0.5371100902557373),
 ('superb', 0.4901832044124603),
 ('impressive', 0.48177313804626465),
 ('fabulous', 0.480276882648468),
 ('wonderful', 0.47941309213638306),
 ('bad', 0.4766196608543396),
 ('awesome', 0.47208553552627563),
 ('top_notch', 0.4645337462425232),
 ('exceptional', 0.4581737518310547),
 ('alright', 0.4473245143890381),
 ('well_constructed', 0.44019460678100586),
 ('hefty', 0.43357741832733154),
 ('goodthe', 0.43194079399108887),
 ('reasonable', 0.42979174852371216)]

In [23]:
w2v_model.most_similar(negative=['bad'], topn=20)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('edition', 0.2583009600639343),
 ('fl', 0.2517528235912323),
 ('chefs_knives', 0.24953234195709229),
 ('supply', 0.24472060799598694),
 ('graduate', 0.24192315340042114),
 ('photographer', 0.2396591156721115),
 ('12inch', 0.23835378885269165),
 ('recommends', 0.23506072163581848),
 ('48', 0.2341330647468567),
 ('labs', 0.23215271532535553),
 ('chef_knife', 0.23086333274841309),
 ('42', 0.228726327419281),
 ('vs', 0.22734788060188293),
 ('recently_purchased', 0.2271212339401245),
 ('runner', 0.2265326976776123),
 ('tablecloth', 0.22239704430103302),
 ('stuffer', 0.22112849354743958),
 ('saucier', 0.21801993250846863),
 ('stainlesssteel', 0.21706584095954895),
 ('26', 0.2165238857269287)]

The results demonstrate how effective Word Vectors are in understanding context between words. The result show that this algorithm is easily able to identify words that are based on similar concepts, even though they are less likely to appear in the same sentences. 

The most similar word with “good” is “great”, and the next are decent, fantastic, excellent, nice and so on. The result also shows that “bad” is similar with “good”. The reason for this is because these words are more likely to have similar labels, which forces their vectors to train into values that can predict those labels correctly.

The most dissimilar words with “bad” are shown in the list: “edition”, “fl”, “chefs_knives”… They all have no relationship with the target word “bad”.


# References:
1. https://radimrehurek.com/gensim/models/word2vec.html
2. https://radimrehurek.com/gensim/models/phrases.html
3. https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial
